In [ ]:

from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime, Text, event
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

Base = declarative_base()

# --- MODELOS ---

class Usuario(Base):
    __tablename__ = 'usuarios'
    id = Column(Integer, primary_key=True)
    nombre = Column(String, nullable=False)
    email = Column(String, unique=True, nullable=False)
    direccion = Column(Text)
    fecha_creacion = Column(DateTime, default=datetime.utcnow)

    pedidos = relationship('Pedido', back_populates='usuario')


class Producto(Base):
    __tablename__ = 'productos'
    id = Column(Integer, primary_key=True)
    nombre = Column(String, nullable=False)
    descripcion = Column(Text)
    precio = Column(Float, nullable=False)
    stock = Column(Integer, nullable=False)

    items = relationship('OrderItem', back_populates='producto')


class Pedido(Base):
    __tablename__ = 'pedidos'
    id = Column(Integer, primary_key=True)
    usuario_id = Column(Integer, ForeignKey('usuarios.id'), nullable=False)
    fecha = Column(DateTime, default=datetime.utcnow)
    estado = Column(String)

    usuario = relationship('Usuario', back_populates='pedidos')
    items = relationship('OrderItem', back_populates='pedido')

    def total(self):
        return sum(item.cantidad * item.precio_unitario for item in self.items)

    def detalle(self):
        lineas = []
        for item in self.items:
            nombre = item.producto.nombre if item.producto else "Producto eliminado"
            subtotal = item.cantidad * item.precio_unitario
            lineas.append(f"{item.cantidad} x {nombre} @ {item.precio_unitario:.2f} = {subtotal:.2f}")
        return lineas

    def marcar_como_enviado(self):
        if self.estado != 'enviado':
            for item in self.items:
                if item.producto:
                    item.producto.stock -= item.cantidad
            self.estado = 'enviado'


class OrderItem(Base):
    __tablename__ = 'order_items'
    id = Column(Integer, primary_key=True)
    pedido_id = Column(Integer, ForeignKey('pedidos.id'), nullable=False)
    producto_id = Column(Integer, ForeignKey('productos.id', ondelete='SET NULL'), nullable=True)
    cantidad = Column(Integer, nullable=False)
    precio_unitario = Column(Float, nullable=False)

    pedido = relationship('Pedido', back_populates='items')
    producto = relationship('Producto', back_populates='items', passive_deletes=True)


# --- CONFIGURACIÓN DE LA BASE DE DATOS (SQLite) ---

engine = create_engine('sqlite:///:memory:', echo=False)

# Activa las claves foráneas en SQLite
@event.listens_for(engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)


ok veo que si elimino un producto no se elimina el pedido que contiene el producto pero en el item pierdo los datos. habra una forma de guardar esa informacion o la solucion es no poder eliminar el producto que crees que seria lo mejor?

ChatGPT dijo:
Buena observación. Sí, si eliminás un producto y usás producto_id con ondelete='SET NULL', el pedido se conserva, pero perdés acceso a la información del producto (nombre, precio, etc.) en ese OrderItem. Por eso, hay tres caminos posibles, y te explico los pros y contras de cada uno:

- ✅ OPCIÓN 1: Guardar una copia de los datos del producto en OrderItem
Cómo: Al crear un OrderItem, copiás el nombre y el precio_unitario directamente.

```python
class OrderItem(Base):
    __tablename__ = 'order_items'
    id = Column(Integer, primary_key=True)
    pedido_id = Column(Integer, ForeignKey('pedidos.id'), nullable=False)
    producto_id = Column(Integer, ForeignKey('productos.id', ondelete='SET NULL'), nullable=True)
    cantidad = Column(Integer, nullable=False)
    precio_unitario = Column(Float, nullable=False)
    nombre_producto = Column(String, nullable=False)  # copia del nombre

    pedido = relationship('Pedido', back_populates='items')
    producto = relationship('Producto', back_populates='items', passive_deletes=True)
```


Y al crear un OrderItem, guardás también nombre_producto = producto.nombre.

- Ventajas:
  - Conservás los datos importantes del producto aunque se elimine.
  - Podés permitir el borrado de productos sin romper pedidos viejos.

- Desventajas:
  - Redundancia: si cambia el nombre del producto, no se refleja en los ítems ya creados.
  - Mayor mantenimiento, pero es común en apps reales.

- ✅ OPCIÓN 2: No permitir borrar productos que tienen pedidos

Cómo: Verificás si un producto tiene OrderItems antes de borrarlo.

- Ventajas:
  - Relación 100% consistente, sin duplicar datos.

- Desventajas:
  - No podés borrar productos que ya se usaron. Solo podrías marcarlos como "inactivos".

- ✅ OPCIÓN 3: Eliminar el producto pero mantenerlo como “fantasma”

- Cómo: En lugar de borrarlo de la base, lo marcás como eliminado:

    ```python
    class Producto(Base):
        __tablename__ = 'productos'
        id = Column(Integer, primary_key=True)
        nombre = Column(String, nullable=False)
        eliminado = Column(Boolean, default=False)
        # resto igual
    ```
    
Y simplemente filtrás por eliminado=False en la app.

- Ventajas:
  - No se pierden datos.
  - Flexible para restaurar o auditar.
  - Sin duplicación.

- Desventajas:
  - Más código para filtrar productos activos.

🏁 ¿Cuál es la mejor?

Depende del contexto, pero la opción 1 (copiar nombre y precio en OrderItem) es la más usada si querés mantener trazabilidad real de lo que se vendió en ese momento, incluso si el producto fue eliminado o cambió de nombre/precio.

¿Querés que te modifique el código para aplicar esta opción?